In [1]:
import xgboost as xgb
from pickle import dump, load
from utils import Dataset

In [2]:
hp = {
    
}

In [3]:
with open("data/train_ds.pkl", "rb") as f:
    ds = load(f)

X, y, names = ds.get_X(), ds.get_y(), ds.get_feature_names()

AttributeError: 'Dataset' object has no attribute 'get_names'

In [ ]:
model = xgb.XGBClassifier(objective="multi:softmax")

In [ ]:
model = model.fit(X, y, verbose=10000)

In [ ]:
with open("models/first_try", "wb") as f:
    dump(model, f)